# Model Eğitimi ve Karşılaştırma

Bu notebook, phishing tespit projesi için çeşitli makine öğrenmesi modellerini eğitir, karşılaştırır ve en iyi modeli seçer.

## İçerik:
1. **Veri Yükleme** - İşlenmiş train/test verilerini yüklenecek
2. **Model Eğitimi** - 4 farklı modeli eğitilecek
3. **Model Karşılaştırması** - Performanslar karşılaştırılacaktır
4. **En İyi Model Seçimi** - En iyi model ve diğer modeller kaydedilecek
5. **Sonuçlar** - Eğitim özeti

In [1]:
# Gerekli kütüphaneleri import et
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import sys

warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Proje kök dizinini Python path'e ekle
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Config ve modülleri import et
from src import config
from src.model_training import (
    train_all_models,
    compare_models,
    save_best_model,
    save_all_models,
    get_model_summary,
    print_model_summary
)

print("✅ Kütüphaneler başarıyla yüklendi!")
print(f"📁 Proje Dizini: {config.BASE_DIR}")
print(f"📁 Model Dizini: {config.MODELS_DIR}")

✅ Kütüphaneler başarıyla yüklendi!
📁 Proje Dizini: c:\Users\PC\Desktop\classification_project
📁 Model Dizini: c:\Users\PC\Desktop\classification_project\models


## 1. Veri Yükleme

Data Preparation aşamasında hazırlanan train ve test verileri yüklenecek.

**Beklenen Veri:**
- Train: 4,679 satır x 28 sütun (27 özellik + 1 hedef)
- Test: 1,170 satır x 28 sütun (27 özellik + 1 hedef)
- Çıkarılan özellikler: `double_slash_redirecting`, `popUpWidnow`, `port`

In [2]:
print("="*80)
print("VERİ YÜKLEME")
print("="*80)

try:
    # Train ve test verilerini yükle
    train_data = pd.read_csv(config.TRAIN_DATA_FILE)
    test_data = pd.read_csv(config.TEST_DATA_FILE)
    
    print(f"\n✅ Veriler başarıyla yüklendi!")
    print(f"   • Train: {config.TRAIN_DATA_FILE.name}")
    print(f"   • Test:  {config.TEST_DATA_FILE.name}")
    
    # Özellik listesini yükle
    with open(config.PROCESSED_DATA_DIR / "feature_names.txt", 'r', encoding='utf-8') as f:
        feature_names = [line.strip() for line in f.readlines()]
    
    print(f"\n📊 Veri Boyutları:")
    print(f"   • Train: {train_data.shape[0]:,} satır x {train_data.shape[1]} sütun")
    print(f"   • Test:  {test_data.shape[0]:,} satır x {test_data.shape[1]} sütun")
    print(f"   • Özellik Sayısı: {len(feature_names)}")
    
    # Özellik ve hedef değişkeni ayır
    X_train = train_data[feature_names]
    y_train = train_data[config.TARGET_NAME]
    X_test = test_data[feature_names]
    y_test = test_data[config.TARGET_NAME]
    
    print(f"\n📋 Sınıf Dağılımları:")
    print(f"   Train:")
    train_dist = y_train.value_counts().sort_index()
    for cls, count in train_dist.items():
        label = config.CLASS_LABELS[cls]
        percentage = (count / len(y_train)) * 100
        print(f"      • {label} ({cls:2}): {count:,} ({percentage:.2f}%)")
    
    print(f"   Test:")
    test_dist = y_test.value_counts().sort_index()
    for cls, count in test_dist.items():
        label = config.CLASS_LABELS[cls]
        percentage = (count / len(y_test)) * 100
        print(f"      • {label} ({cls:2}): {count:,} ({percentage:.2f}%)")
    
    print(f"\n✅ Veri hazırlama tamamlandı!")
    
except Exception as e:
    print(f"❌ Hata: {e}")
    raise

VERİ YÜKLEME

✅ Veriler başarıyla yüklendi!
   • Train: train.csv
   • Test:  test.csv

📊 Veri Boyutları:
   • Train: 4,679 satır x 28 sütun
   • Test:  1,170 satır x 28 sütun
   • Özellik Sayısı: 27

📋 Sınıf Dağılımları:
   Train:
      • Phishing (-1): 2,415 (51.61%)
      • Legitimate ( 1): 2,264 (48.39%)
   Test:
      • Phishing (-1): 604 (51.62%)
      • Legitimate ( 1): 566 (48.38%)

✅ Veri hazırlama tamamlandı!


In [3]:
print("\n" + "="*80)
print("TÜM MODELLERİN EĞİTİMLERİ")
print("="*80)

# Tüm modelleri eğit
all_results = train_all_models(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    config=config,
    verbose=False
)

print("\n" + "="*80)
print("MODEL EĞİTİMİ TAMAMLANDI")
print("="*80)
print(f"✅ {len(all_results)} model başarıyla eğitildi!")


TÜM MODELLERİN EĞİTİMLERİ

MODEL EĞİTİMİ TAMAMLANDI
✅ 4 model başarıyla eğitildi!


In [4]:
print("\n" + "="*80)
print("TÜM MODELLERİ KAYDETME")
print("="*80)

# Tüm modelleri kaydet
save_all_models(
    all_results=all_results,
    models_dir=config.MODELS_DIR,
    verbose=True
)


TÜM MODELLERİ KAYDETME

TÜM MODELLERİ KAYDETME
✅ Random Forest             → random_forest.pkl
✅ Decision Tree             → decision_tree.pkl
✅ Gradient Boosting         → gradient_boosting.pkl
✅ Logistic Regression       → logistic_regression.pkl

✅ 4 model başarıyla kaydedildi!
📁 Kayıt Dizini: c:\Users\PC\Desktop\classification_project\models


In [5]:
# ============================================================================
# AUTOML - EN BASİT HALİ
# ============================================================================

print("\n" + "="*80)
print("AUTOML")
print("="*80)

from src.automl import train_automl, save_automl_model, compare_with_traditional_models

# AutoML çalıştır (3 dakika)
automl_results = train_automl(X_train, y_train, X_test, y_test, time_limit=180)

# Kaydet
save_automl_model(automl_results, config.MODELS_DIR / "automl.pkl")

# Karşılaştır
comparison = compare_with_traditional_models(automl_results, all_results)

# CSV kaydet
comparison.to_csv(config.REPORTS_DIR / "automl_comparison.csv", index=False)

print("\n✅ Tamam, bitti!")


AUTOML


	Failed to import torch or check CUDA availability!Please ensure you have the correct version of PyTorch installed by running `pip install -U torch`


AUTOML BAŞLIYOR (AutoGluon)
⏱️ Süre: 180 saniye (3.0 dakika)
🚀 Eğitim başladı...


		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==1.4.0`.
	Failed to import torch or check CUDA availability!Please ensure you have the correct version of PyTorch installed by running `pip install -U torch`
		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==1.4.0`.
	Failed to import torch or check CUDA availability!Please ensure you have the correct version of PyTorch installed by running `pip install -U torch`
	Failed to import torch or check CUDA availability!Please ensure you have the correct version of PyTorch installed by running `pip install -U torch`
	Failed to import torch or check CUDA availability!Please ensure you have the correct version of PyTorch installed by running `pip install -U torch`
		`import catboost` failed. A quick tip is to install via `pip install autogluon.tabular[catboost]==1.4.0`.
	Failed to import torch or check CUDA availability!Please ensure you have the co

✅ Bitti!

📊 Sonuçlar:
   Accuracy:  0.9496
   F1-Score:  0.9477
   ROC-AUC:   0.9912

💾 Model kaydedildi: AutogluonModels/
   En iyi: WeightedEnsemble_L2

KARŞILAŞTIRMA
              Model  F1-Score  Accuracy  ROC-AUC
          AutoGluon    0.9477    0.9496   0.9912
      Random Forest    0.0000    0.0000   0.0000
      Decision Tree    0.0000    0.0000   0.0000
  Gradient Boosting    0.0000    0.0000   0.0000
Logistic Regression    0.0000    0.0000   0.0000

🏆 Kazanan: AutoGluon

✅ Tamam, bitti!
